# Bruker Data Grepping and Alignment Notebook
### Jeremy Delahanty June 2021

Intended to grep different files/projects/datasets from user input and retain them for use in analysis/display later. The lack of unified filenaming structures between projects will break the code... A convetion of XXX### for animal names has been established for Austin's projects

In [148]:
# Import packages
from pathlib import Path
import pathlib
import glob
import re
import json
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd


In [149]:
lab_basepath = "Y:/"
# project_dict = {"specialk": ["learned_helplessness", "chronic_mild_stress"]}

In [150]:
def grep_teams(team_selection=[], lab_basepath="Y:/"):
    """
    Grabs team list from server based on user's input.
    
    User can define which teams they want to use for their analyses and
    the function will glob the paths for their selection.
    
    Parameters
    ----------
    arg1: list
        List of strings for teams of interest
        Default is empty list
    arg2: string
        Basepath for server location on machine
        Default is Y:/ for mapped Windows drive
    
    
    Returns
    -------
    1. list
        List of team path grabbed from server successfully
    2. list
        List of teams not found
    """

    # Take basepath and glob all available files and directories
    team_glob = Path(lab_basepath).glob("{}".format("*"))
    
    # Check if no team was specifically asked for, tell user we're gathering all teams
    if team_selection == []:

        print("Gathering all teams...")

        # List comprehension for returning all directories in Tye Lab server
        team_list = [team for team in team_glob if team.is_dir()]

    else:
        
        # List comprehension for returning only directories user wants in the Tye Lab server 
        team_list = [team for team in team_glob if team.name in team_selection and team.is_dir()]
    
    # Create temporary list for checking if selected teams exist
    tmp = []

    # For the teams that were globbed successfully, append the team to the temp list
    for globbed_team in team_list:
        tmp.append(globbed_team.name)
    
    # Compare team selection with returned teams using sets, convert to list
    missing_teams = list(set(team_selection) - set(tmp))
    
    # If the missing_teams list is empty, the program found all requested teams
    if missing_teams == []:
        print("Found All Selected Teams")
    
    # Else, some teams weren't found. Tell the user which teams weren't found.
    else:
        print("Failed to find team(s):", missing_teams)
   
    # Show user which teams were returned
    print("Teams Returned:")
    for team in team_list:
        print("{} ".format(team.name))
    
    # Return the list of projects gathered
    return team_list, missing_teams

team_list, missing_teams = grep_teams(["specialk"])

Found All Selected Teams
Teams Returned:
specialk 


In [151]:
def choose_projects(team_list, project_selection={}):
    """
    Generates project paths list based on user's selection.
    
    User can define which project they want to use for their analyses and
    this function generates the paths for their selection.
    
    Parameters
    ----------
    arg1: list
        List of strings for teams of interest from grep_teams()
    arg2: dict
        Dictionary of values that will be used to create specific
        paths for selected teams and their projects
    
    Returns
    -------
    1. list
        List of team/project Paths to grep in next steps
    """

    # Make dictionary using the teams in team_list as keys
    project_dict = {team: [] for team in team_list}

    # For each time in the team_list, append the Path name's project's values
    for team in team_list:
        project_dict[team].append(project_selection[team.name])
    
    # Make empty project list
    project_dir_list = []

    for team in project_dict.keys():
        for project in range(len(project_dict[team][0])):
            project_dir_list.append(team / project_dict[team][0][project])
    
    print("Returned Directories: ")

    for directory in project_dir_list:
        print(directory)
    
    return project_dir_list
    
project_list = choose_projects(team_list, project_selection={"specialk": ["learned_helplessness"]})

Returned Directories: 
Y:\specialk\learned_helplessness


In [153]:
def choose_animals(project_list, animal_group="all"):
    """
    Generates animal paths list based on user's selection.
    
    User can define which cohort of animals they want to use 
    for their analyses. This function generates the paths for 
    their selection that meet specified conditions.
    
    Parameters
    ----------
    arg1: list
        List of strings for projects of interest from choose_projects()
    arg2: str
        String of value for which animal paths to gather.
        Default value is all.
    
    Returns
    -------
    1. list
        List of team/project/animal Paths to grep in next steps
    """
    
    # Create empty animal list for path generation
    animal_list = []
    
    # If the animal group is left as default/specified as all, grab all animals
    if animal_group == "all":
        print("Grabbing all animals...")
        
        # For each project directory in the project list
        for project_dir in project_list:
            
            # For each animal globbed in the project directory
            for animal in project_dir.glob("*"):
                
                # Append the animal's path to the animal_list
                print(project_dir.name, animal.name)
                animal_list.append(animal)
    
    # Else, only select animals from the specified group
    else:
        print("Grabbing only {} animals...".format(animal_group))

        # Format the animal group with the user's input
        animal_group = "[A-Z]{2}" + animal_group + "\d{3}"
        
        # For each project_directory in project_list
        for project_dir in project_list:

            # For each animal globbed in project directory
            for animal in project_dir.glob("*"):
                
                # Use regex to grab only the requested animal
                r = re.search(animal_group, string=animal.name)
                
                # If the search returns None, the animal didn't match the request
                # Skip over it with pass.
                if r is None:
                    pass
                
                # If something is returned, take the match object's value and append
                # the animal to the project directory.
                else:
                    print(project_dir.name, r.group(0))
                    animal_list.append(project_dir / r.group(0))
    
    # Finally, return the list of animals
    return animal_list

animal_list = choose_animals(project_list, animal_group="E")

Grabbing only E animals...
learned_helplessness LHE011
learned_helplessness LHE012
learned_helplessness LHE013
learned_helplessness LHE014
learned_helplessness LHE015
learned_helplessness LHE016


In [154]:
def choose_data(animal_list, data_group=[], verbose=True):
    """
    Generates animal's data paths list based on user's selection.
    
    User can define which dataset to use for the animals they 
    want to use for their analyses. This function generates the
    paths for their selection that meet specified conditions.
    
    Parameters
    ----------
    arg1: list
        List of paths for animals of interest from choose_animals()
    arg2: list
        List of strings for which datasets to gather.
        Default value is all.
    arg3: bool
        Boolean argument for verbose output of paths found or
        not found by the function. Default is True.
    
    Returns
    -------
    1. list
        List of team/project/animal/dataset Paths to grep in
        next steps
    """
    
    # Create empty data list for path generation
    data_list = []
    
    # If data_group is left as default or specified as empty,
    # grab all folders
    if data_group == []:
        print("Grabbing all data folders...")
        
        # For each animal in the animal list
        for animal in animal_list:
            
            # For the data_dir in the globbed animal_path
            for data_dir in animal.glob("*"):
                
                # Append the data_dir to the data_list
                print("Grabbing", animal.name, data_dir.name)
                data_list.append(data)
    
    #TODO: Make verbose into its own function
    elif len(data_group) > 0 and verbose is True:
        print("Grabbing...")
        for data_type in data_group:
            print(data_type, "data")

        print("\nFrom Projects(s)...")
        project_list = list(set([project.parent.name for project in animal_list]))
        for project in project_list:
            print(project)
            
        print("\nIn Team(s)...")
        team_list = list(set([team.parent.parent.name for team in animal_list]))
        for team in team_list:
            print(team)
        print("\nFor Animals...")
        for animal in animal_list:
            print(animal.name)
        
        print("\nChecking for directories...")
        for animal in animal_list:
            for data_type in data_group:
                if (animal / data_type).is_dir():
                    print("Found", animal.name, data_type)
                    data_list.append(animal / data_type)
                else:
                    print("Not Found!", animal.name, data_type)
    else:
       
        #TODO: Write a function for checking
        print("Grabbing specified directories...\n")
        
        for animal in animal_list:
            for data_type in data_group:
                if (animal / data_type).is_dir():
                    data_list.append(animal / data_type)
                else:
                    print(animal.name, data_type, "Not Found!")
    
    # Tell user which directories were returned
    print("\nReturning Directories:")
    for data_dir in data_list:
        print(data_dir)

    return data_list
                

data_list = choose_data(animal_list, data_group=["twop"], verbose=True)

Grabbing...
twop data

From Projects(s)...
learned_helplessness

In Team(s)...
specialk

For Animals...
LHE011
LHE012
LHE013
LHE014
LHE015
LHE016

Checking for directories...
Found LHE011 twop
Found LHE012 twop
Found LHE013 twop
Found LHE014 twop
Found LHE015 twop
Found LHE016 twop

Returning Directories:
Y:\specialk\learned_helplessness\LHE011\twop
Y:\specialk\learned_helplessness\LHE012\twop
Y:\specialk\learned_helplessness\LHE013\twop
Y:\specialk\learned_helplessness\LHE014\twop
Y:\specialk\learned_helplessness\LHE015\twop
Y:\specialk\learned_helplessness\LHE016\twop


In [200]:
def grep_twop_behavior_raw(data_list):
    """
    Generates animal's raw data paths list based on user's selection
    
    Parameters
    ----------
    arg1: list
        List of paths for animals of interest from choose_data()
    
    Returns
    -------
    1. list
        List of 2P raw behavior data Paths
    """

    twop_raw_beh_list = []
    
    
    for directory in data_list:
        search = directory.glob("*/*raw_behavior*/*.csv")
        for result in search:
            twop_raw_beh_list.append(result)
            
    return twop_raw_beh_list

twop_raw_beh_list = grep_twop_behavior_raw(data_list)

In [201]:
def grep_twop_behavior_config(data_list):
    """
    Generates animal's configuration paths list based on user's selection
    
    Parameters
    ----------
    arg1: list
        List of paths for animals of interest from choose_data()
    
    Returns
    -------
    1. list
        List of 2P behavior configuration Paths
    """
    
    twop_config_list = []
    
    #TODO: Use .parents instead of parent.parent
    for directory in data_list:
        search = directory.glob("*/*.json")
        for result in search:
            twop_config_list.append(result)
            
    return twop_config_list
            
twop_config_list = grep_twop_behavior_config(data_list)

In [204]:
def merge_2p_behavior(twop_raw_beh_list, twop_config_list, twop_microscopy_list=[]):
    """
    Merges twop behavior and configuration files' data together
    
    Parameters
    ----------
    arg1: list
        List of paths for raw behavior data
    arg2: list
        List of paths for configration files
    
    Returns
    -------
    1. list
        List of merged 2P behavior paths
    """

    # TODO: Use Kyle's code to align these files with the relative timestamps of microscopy
    # TODO: Get zipping of these two lists to work so finding config file is already completed...
    # TODO: Make force overwrite/recompile the different datasets
    # TODO: Make verbose version of this function
    needs_merging = []
    
    merged_list = []
    
    # Give name_date pattern for the files we're aligning
    name_date_pattern = re.compile("\d{8}_[A-Z]{3}\d{3}")
    
    # First, check for cleaned data
    for directory in twop_raw_beh_list:
        
        # Glob for the aligned json file
        cleaned_check = directory.parents[1].glob("*_merged.json")
        
        # Make a list using the result of the glob using list comprehension
        clean_checklist = [session for session in cleaned_check]
        
        # If the list is empty, the session needs merging
        if len(clean_checklist) == 0:
            
            # Show which session needs alignment and append the directory to needs_merging list
            print("Session Needs Merging:", directory.parents[3].name, directory.parents[1].name)
            needs_merging.append(directory)
        
        # Else, the session has already been merged.  Append the filepath to the merged_list.
        else:
            merged_file = clean_checklist[0]
            print("Session already merged:", merged_file)
            merged_list.append(merged_file)
    
    # Now, do the merging
    for raw_file in needs_merging:
        print("Merging:", raw_file.name)
        
        merged = {}
        
        parent_folder = raw_file.parents[1]
        
        raw_behavior_df = pd.read_csv(raw_file, index_col="Time(ms)").rename(columns=lambda col:col.strip())
        
        # Any value below 3V is not signal, turn it to zero by filtering
        # values so all that remains are values greater than 3. All else
        # will be 0.
        raw_behavior_df = raw_behavior_df > 3
        
        # Convert all values to int; is necessary for pd.df.diff() to produce
        # negative values used for stop times of each event
        raw_behavior_df = raw_behavior_df.astype(int)
        
        # Take the diff of each column; gives start and stop of each signal
        raw_behavior_df = raw_behavior_df.diff()

        # Replace any NaN values with 0
        raw_behavior_df = raw_behavior_df.fillna(0)
        
        # Grab start and stop values for licks
        merged["LickOn"] = raw_behavior_df[raw_behavior_df["Lick"] == 1].index.tolist()
        merged["LickOff"] = raw_behavior_df[raw_behavior_df["Lick"] == -1].index.tolist()
        
        # Grab start and stop values for Airpuff Solenoid
        merged["AirpuffOn"] = raw_behavior_df[raw_behavior_df["Airpuff"] == 1].index.tolist()
        merged["AirpuffOff"] = raw_behavior_df[raw_behavior_df["Airpuff"] == -1].index.tolist()
        
        # Grab start and stop values for Liquid Solenoid
        merged["LiquidOn"] = raw_behavior_df[raw_behavior_df["Liquid"] == 1].index.tolist()
        merged["LiquidOff"] = raw_behavior_df[raw_behavior_df["Liquid"] == -1].index.tolist()
        
        # Grab start and stop values for Speaker
        merged["SpeakerOn"] = raw_behavior_df[raw_behavior_df["Speaker"] == 1].index.tolist()
        merged["SpeakerOff"] = raw_behavior_df[raw_behavior_df["Speaker"] == -1].index.tolist()
        
        #TODO: This should be just one regex, not sure why complete isn't working...
        
        # Perform the regex for the name and date of the file
        r_name_date = re.search(pattern=name_date_pattern, string=raw_file.name)
        
        # Give the pattern for the plane of interest
        plane_pattern = "_plane\d{1}"
        
        # Perform the regex for the plane number of the file
        r_plane = re.search(pattern=plane_pattern, string=raw_file.name)
        
        # Concatenate strings into final merged file as json type
        merged_name = r_name_date.group(0) + r_plane.group(0) + "_merged.json"
        
        # Append the parent folder with this name to create the file later
        merged_filename = parent_folder / merged_name
        
        # Grab the config file for this plane from the parent folder
        config_glob = parent_folder.glob("*.json")
        
        # Config gather the config file result in a list
        config_file_result = [config for config in config_glob]
        
        # The config file is the only element of this list, not sure how to retain only the relevant file without lists...
        config_file = config_file_result[0]
        
        # Open the json file using json package 
        with open(config_file, "r") as inFile:
            
            # The configuration is the read file
            config = inFile.read()
            
            # Load the contents of the config with json.loads()
            config_contents = json.loads(config)
            
            # Gather the trial types from the configuration
            trial_types = config_contents["trialArray"]
        
        # Append trial types to aligned_file
        merged["trialTypes"] = trial_types

        # Create the new file using json package
        with open(merged_filename, "w") as outFile:
            
            # Use json.dump to write aligned_dictionary to file
            json.dump(merged, outFile)
        
        # Tell user the file has been written
        print("Written", merged_filename)
        aligned_list.append(merged_filename)
        
    return merged_list

merged_list = merge_2p_behavior(twop_raw_beh_list, twop_config_list)

Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210603\20210603_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210604\20210604_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210605\20210605_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210607\20210607_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210608\20210608_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210611\20210611_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE011\twop\20210612\20210612_LHE011_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE012\twop\20210603\20210603_LHE012_plane0_merged.json
Session already merged: Y:\specialk\learned_helplessness\LHE012\twop\20210604\20210604_LHE012_pl

In [206]:
def align_2p_behavior(merged_list, alignment_positions=["LiquidOn", "SpeakerOn"], window=5):
    """
    Aligns twop behavior to different positions from user within specific window
    
    Parameters
    ----------
    arg1: list
        List of paths for merged behavior data
    arg2: list
        Alignment positions to store
    arg3: int
        Window (in seconds) for pre/post event alignment
    
    Returns
    -------
    1. list
        List of aligned 2P behavior paths
    """

    needs_alignment = []
    
    aligned_list = []
    
    # First, check for cleaned data
    for directory in merged_list:
        
        # Glob for the aligned json file
        cleaned_check = directory.parents[0].glob("*aligned_*")
        
        # Make a list using the result of the glob using list comprehension
        clean_checklist = [session for session in cleaned_check]
        
        # If the list is empty, the session needs alignment
        if len(clean_checklist) == 0:
            
            # Show which session needs alignment and append the directory to needs_merging list
            print("Session Needs Alignment:", directory.parents[2].name, directory.parents[0].name)
            needs_alignment.append(directory)
        
        # Else, the session has already been aligned.  Append the filepath to the aligned_list.
        else:
            aligned_file = clean_checklist[0]
            print("Session already aligned:", aligned_file)
            aligned_list.append(aligned_file)
    
    window = window * 1000

    for merged_file in needs_alignment:

        with open(merged_file, "r") as inFile:

            contents = inFile.read()

            timestamps = json.loads(contents)

        trial_df = pd.DataFrame()
        
        lick_timestamps = []
        liquid_counter = 0
        airpuff_counter = 0

        # TODO: Each segment should be its own function call ie liquidon, speakers, airpuffon
        for (index, trial) in enumerate(timestamps["trialTypes"]):
            if trial == 1:
                liquid_counter += 1
                trial_name = "Trial_" + str(index + 1)
                trial_type = "Sucrose_" + str(liquid_counter)
                s = pd.Series(trial_type, dtype=str, name=trial_name)
                trial_df = trial_df.append(s)
            else:
                airpuff_counter += 1
                trial_name = "Trial_" + str(index + 1)
                trial_type = "Airpuff_" + str(airpuff_counter)
                s = pd.Series(trial_type, dtype=str, name=trial_name)
                trial_df = trial_df.append(s)
        
        trial_df.index.name = "trial"
        trial_df.columns = ["trial_type"]
        
        exclude_keys = ["LickOn", "LickOff", "LiquidOff", "AirpuffOff", "trialTypes"]
        
        for key in timestamps.keys():
            if key not in exclude_keys:
                trial_df[key] = np.nan
        
        speakeron_df = pd.DataFrame()
        
        for (index, trial) in enumerate(timestamps["SpeakerOn"]):
            trial_number = "Trial_" + str(index + 1)
            speaker_on_ms = trial
            s = pd.Series(speaker_on_ms, name=trial_number)
            speakeron_df = speakeron_df.append(s)
        
        speakeron_df.index.name = "trial"
        speakeron_df.columns = ["SpeakerOn"]
        
        trial_df.update(speakeron_df)
        
        speakeroff_df = pd.DataFrame()
        
        for (index, trial) in enumerate(timestamps["SpeakerOff"]):
            
            trial_number = "Trial_" + str(index + 1)
            speaker_off_ms = trial
            s = pd.Series(speaker_off_ms, dtype=int, name=trial_number)
            speakeroff_df = speakeroff_df.append(s)
        
        speakeroff_df.index.name = "trial"
        speakeroff_df.columns = ["SpeakerOff"]
        
        trial_df.update(speakeroff_df)
        
        trial_df.reset_index(inplace=True)
        trial_df.set_index("trial_type", inplace=True)

        
        liquid_df = pd.DataFrame()

        for (index, trial) in enumerate(timestamps["LiquidOn"]):
            
            liquid_trial = "Sucrose_" + str(index + 1)
            liquid_start_ms = trial
            s = pd.Series(liquid_start_ms, dtype=int, name=liquid_trial)
            liquid_df = liquid_df.append(s)
        
        liquid_df.index.name = "trial_type"
        liquid_df.columns = ["LiquidOn"]
        
        trial_df.update(liquid_df)
        
        airpuff_df = pd.DataFrame()

        for (index, trial) in enumerate(timestamps["AirpuffOn"]):
            airpuff_trial = "Airpuff_" + str(index + 1)
            airpuff_start_ms = trial
            s = pd.Series(airpuff_start_ms, dtype=int, name=airpuff_trial)
            airpuff_df = airpuff_df.append(s)
        
        airpuff_df.index.name = "trial_type"
        
        if airpuff_df.size > 0:
            airpuff_df.columns = ["AirpuffOn"]
            trial_df.update(airpuff_df)
        else:
            pass
        
        trial_df.reset_index(inplace=True)
        trial_df.set_index("trial", inplace=True)

                
        # Use list comprehension for lick timestamps
        lick_timestamps = [lick for (index, lick) in enumerate(timestamps["LickOn"])]
        
        # Create columns for aligning licks to different events and then create lick lists
        # This is terrible and a better way to assign licks to a list should be found, but
        # without using a relational system I'm not sure how I can alter this...
        # TODO: Make this a function that is iterated over instead of multiple for loops like this...
        # Possible solutions: Create ITI starts/stops as dataframe values and evaluate against range
        
        for target in alignment_positions:
            column = target + "_licks"
            trial_df[column] = np.empty((len(trial_df), 0)).tolist()
            for lick_time in lick_timestamps:
                for value in trial_df[target].items():
                    if (value[1] - window) <= lick_time <= (value[1] + window):
                        tmp = value[0]
                        trial_df[column].loc[tmp].append(lick_time)
                    else:
                        pass

        # Create empty lists for ITI licks
        trial_df["ITI_licks"] = np.empty((len(trial_df), 0)).tolist()

        # Get leftover ITI licks by creating sets out of aligned licks
        accounted_licks = []
        for value in trial_df["LiquidOn_licks"].items():
            if len(value[1]) == 0:
                pass
            else:
                for licks in value[1]:
                    accounted_licks.append(licks)
        
        accounted_licks = set(accounted_licks)
        licks_set = set(lick_timestamps) 
        iti_licks = list(licks_set - accounted_licks)


        # Again, a bad way to do this. Too many loops and likely to be quite slow...
        for lick_time in iti_licks:
            for value in trial_df["ITI_licks"].items():
                tmp = value[0]
                tmp2 = value[1]
                if (trial_df["SpeakerOn"].loc[tmp] - window) <= lick_time <= (trial_df["LiquidOn"].loc[tmp] + window):
                    trial_df["ITI_licks"].loc[tmp].append(lick_time)
        
        
        
        # DEPRECATED: Deprecated for now unless told that having precise timings for ITIs is necessary
        # NOTE: Unsure this is necessary, should be able to infer the ITIs without explicitly having
        # having start/stop times for ITIs
#         trial_df["ITI_start"] = np.nan
#         trial_df["ITI_end"] = np.nan

        # Uses .loc for the index and then the column of interest to set actual value of frame
        # not a copy of the dataframe
        # Set first ITI start as 0ms
#         trial_df.loc["Trial_1", "ITI_start"] = 0
        
#         # Set first ITI end as first SpeakerOn value
#         trial_df.loc["Trial_1", "ITI_end"] = trial_df.loc["Trial_1", "SpeakerOn"]
        
#         trial_df.loc[1:len(trial_df), "ITI_start"] = trial_df.LiquidOn[1:len(trial_df)-1]
#         trial_df.loc[1:len(trial_df), "ITI_end"] = trial_df.SpeakerOn[1:len(trial_df)]

        aligned_filename = merged_file.stem
        aligned_filename = aligned_filename.replace("merged", "aligned")
        aligned_filename = aligned_filename + "_" + "window" + str(int(window/1000)) + ".csv"
        
        aligned_file_path = merged_file.parents[0] / aligned_filename
        
        aligned_list.append(aligned_file_path)
        
        trial_df.to_csv(aligned_file_path)

    return aligned_list


aligned_list = align_2p_behavior(merged_list)

Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210603\20210603_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210604\20210604_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210605\20210605_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210607\20210607_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210608\20210608_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210611\20210611_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE011\twop\20210612\20210612_LHE011_plane0_aligned_window5.csv
Session already aligned: Y:\specialk\learned_helplessness\LHE012\twop\20210603\20210603_LHE012_plane0_aligned_window5.csv
Session already aligned: